# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!Powershell.exe -Command type publications.tsv -Head 3

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-07-21	Preliminary assessment of bacterial antibiotic resistance and Candidatus Liberibacter asiaticus titer in three Florida commercial citrus groves	Crop Protection		"de Gracia Coquerel, M., A. McAuley, J. Wegerif, Q. D. Read, N. Chowdhury, K. C. Jeong, J. G. Morris, S. J. Martins, E. M. Goss, and M. S. Ascunce. 2023. Preliminary assessment of bacterial antibiotic resistance and Candidatus Liberibacter asiaticus titer in three Florida commercial citrus groves. Crop Protection 172:106350. DOI: 10.1016/j.cropro.2023.106350."	degracia-et-al-2023	https://doi.org/10.1016/j.cropro.2023.106350
2023-06-12	Use of automated capillary immunoassay for quantification of antibodies in chicken sera against recombinant Salmonella enterica serotype Heidelberg proteins	Journal of Microbiological Methods		"Yeh, H.-Y., J. G. Frye, C. R. Jackson, Q. D. Read, J. E. Line, and A. Hinton. 2023. Use of automated capillary immunoassay for quantificat

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Structural equation modeling in food science,Invited talk,talk-fsmqhru2023,"USDA Agricultural Research Service, Food Scien...",2023-08-02,"Raleigh, North Carolina",NaN,NaN
1,Power analysis: bureaucratic busywork or criti...,Invited talk,talk-iacuc2023,"USDA Agricultural Research Service, FY2023 IAC...",2023-07-25,"Athens, Georgia",NaN,NaN
2,'Boring' for insights in a 28-year sugarcane i...,Conference presentation,talk-agstats2023,AgStats (Conference on Applied Statistics in A...,2023-05-16,"West Lafayette, Indiana",NaN,NaN
3,Reducing food waste from farm to fork to benef...,Invited talk,talk-ncsu2022,"North Carolina State University, Plant & Micro...",2022-04-05,"Raleigh, North Carolina",NaN,NaN
4,Food waste: wicked problem or win-win?,Invited talk,talk-gwu2021,"George Washington University, guest lecture in...",2021-11-09,"Washington, DC",NaN,NaN
5,Reducing food waste to benefit environment and...,Invited talk,talk-duke2020,"Duke University, University Program in Ecology...",2020-10-08,"Durham, North Carolina",NaN,NaN
6,Which supply chain stages should we target to ...,Conference presentation,talk-esaussee2019,Ecological Society of America/U.S. Society for...,2019-08-14,"Louisville, Kentucky",NaN,NaN
7,Measuring geodiversity to explain biodiversity...,Conference presentation,talk-iale2018,International Association of Landscape Ecology,2018-04-09,"Chicago, Illinois",NaN,NaN
8,Intraspecific variation reflects drivers of ro...,Conference presentation,talk-esa2017,Ecological Society of America,2017-08-08,"Portland, Oregon",NaN,NaN
9,A globally replicated experiment shows that lo...,Conference presentation,talk-esa2015,Ecological Society of America,2015-08-11,"Baltimore, Maryland",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

In [5]:
# Fix the issues with publication date (excel)
str(talks['date'])

'0     2023-08-02\n1     2023-07-25\n2     2023-05-16\n3     2022-04-05\n4     2021-11-09\n5     2020-10-08\n6     2019-08-14\n7     2018-04-09\n8     2017-08-08\n9     2015-08-11\n10    2018-10-24\n11    2017-11-15\n12    2017-01-05\n13    2016-09-20\n14    2015-03-01\n15    2014-06-01\n16    2014-01-15\nName: date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [8]:
%%cmd
dir ..\_talks\

Microsoft Windows [Version 10.0.19045.3208]
(c) Microsoft Corporation. All rights reserved.

C:\Users\qdread\Documents\GitHub\qdread.github.io\markdown_generator>dir ..\_talks\
 Volume in drive C has no label.
 Volume Serial Number is 08D0-A2BB

 Directory of C:\Users\qdread\Documents\GitHub\qdread.github.io\_talks

07/31/2023  11:25 AM    <DIR>          .
07/31/2023  11:25 AM    <DIR>          ..
07/31/2023  11:25 AM               309 2014-01-15-talk-ornl2014.md
07/31/2023  11:25 AM               262 2014-06-01-talk-rmbl2014.md
07/31/2023  11:25 AM               262 2015-03-01-talk-nd2015.md
07/31/2023  11:25 AM               363 2015-08-11-talk-esa2015.md
07/31/2023  11:25 AM               342 2016-09-20-talk-msu2016.md
07/31/2023  11:25 AM               299 2017-01-05-talk-neon2017.md
07/31/2023  11:25 AM               324 2017-08-08-talk-esa2017.md
07/31/2023  11:25 AM               287 2017-11-15-talk-idiv2017.md
07/31/2023  11:25 AM               335 2018-04-09-talk-iale2018.md
0